In [ ]:
import mplfinance as mpf
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.constraints import MaxNorm
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense 
from keras.models import load_model
import pandas as pd
import numpy as np
import requests
import io
import os
import datetime
from concurrent.futures import ThreadPoolExecutor
import json
import urllib.request
import pandas as pd
import ccxt
import os
import gc
from backtest_module import *

## Controle Space

In [ ]:
# ai_options
Normalization_File= "False"
Model_FileName= "/UltimeTradingBot/Data/BUY_FIX/tp80_w7_max3min_Model_true_win_model_Re4.h5"
WINDOW_SIZE= 7
PRESSISION=0.0
PERIODE_START="2023-02-09 00:00:00"
PERIODE_END="2023-02-11 00:00:00"

# start_period = datetime.strptime(PERIODE_START, '%d-%m-%Y %H:%M:%S')
# end_period = datetime.strptime(PERIODE_END, '%d-%m-%Y %H:%M:%S')
start_period = pd.Timestamp(PERIODE_START)
end_period = pd.Timestamp(PERIODE_END)


#trading_options:
TAKE_PROFIT= 0.4  
STOP_LOSS=  0.8 
MAX_HOLDING_TIME= 16
PRECISION = 0.0

USE_TRAILING_STOP_LOSS: False
TRAILING_STOP_LOSS= .002     
TRAILING_TAKE_PROFIT= .002  

PAIR_WITH= "USDT"
TRADE_TOTAL= 100 
TRADE_SLOTS= 5

TRADING_FEE= 0.1
  

## Data space

In [ ]:
#################### Files #################

DATABASE_DIR="./database"
INFO_DIR=f"{DATABASE_DIR}/Info"
ALL_BINANCE_TICKERS_LISTFILE=f"{INFO_DIR}/ALL_BINANCE_TICKERS_LISTFILE.json"
ONLY_MY_HALAL_LIST_FILE=f"{INFO_DIR}/ONLY_MY_HALAL_LIST.json"
halal_list_file_path=f"{INFO_DIR}/halal-crypto-list.txt"
BINANCE_KLINES_DATA_DIR=f"{DATABASE_DIR}/OpenBinance/KLINES"
BINANCE_TRADES_DATA_DIR=f"{DATABASE_DIR}/OpenBinance/Trades"
METADATA_FILE=f"{INFO_DIR}/METADATA.csv"
COINGEKO_INFO_FILE=f"{INFO_DIR}/COINGEKO_INFO_FILE.csv"
########### data info ################


In [ ]:
ALL_BINANCE_TICKERS=get_all_binance_tickers()
ALL_USDT_BUSD_PAIRS=extract_pairs(ALL_BINANCE_TICKERS)
WORKING_PAIRS,VOLATILE_USDT_PAIRS ,VOLATILE_BUSD_PAIRS=get_my_halal_list(halal_file=halal_list_file_path)
MY_HALA_DIC=get_my_halal_list(halal_file=halal_list_file_path)
ALL_CIBLED_COINS = list(set([symbol.split('/')[0].lower() for symbol in ALL_USDT_BUSD_PAIRS] ))


In [21]:
MetaData=read_metadata(WORKING_PAIRS)

STMXUSDT launched 1591844400000
MBLUSDT launched 1582293600000
AXSUSDT launched 1604494800000
ASTRUSDT launched 1646047800000
ADXUSDT launched 1635501600000
EPSUSDT launched 1617354000000
SNXUSDT launched 1594296000000
MTLUSDT launched 1566554400000
ZRXUSDT launched 1551326400000
ELFUSDT launched 1631088000000
NEARUSDT launched 1602651600000
ENSUSDT launched 1636527600000
SCRTUSDT launched 1642766400000
NMRUSDT launched 1597838400000
SKLUSDT launched 1606784400000
GRTUSDT launched 1608237000000
MOBUSDT launched 1651226400000
DNTUSDT launched 1604988000000
QUICKUSDT launched 1628856000000
SLPUSDT launched 1619784000000
PORTOUSDT launched 1637064000000
TOMOUSDT launched 1566651600000
ZECUSDT launched 1553140800000
MULTIUSDT launched 1649217600000
FLUXUSDT launched 1639116000000
PYRUSDT launched 1637913600000
GALUSDT launched 1651752000000
SRMUSDT launched 1597152600000
FIROUSDT launched 1611885600000
LTCBUSD launched 1571644800000
ARBUSD launched 1620993600000
SNMBUSD launched 1661500800

In [28]:
from lightfn_module import *
def get_backtest_historical(exchange, symbol, timeframe, path="database/"):
    symbol = symbol.replace('/','-')
    df = pd.read_csv(filepath_or_buffer=path+"DataBackTest"+"/"+timeframe+"/"+symbol+".csv")
    df = df.set_index(df['date'])
    df.index = pd.to_datetime(df.index, unit='ms')
    del df['date']
    return df


In [43]:
import gc
from collections import defaultdict
from functools import partial
import ccxt

def get_historical_dataframes(working_pairs, oldest_pair="BTC/USDT", timeframes=("1m", "1d", "1h", "5m", "15m"), path="./database/"):
    if oldest_pair not in working_pairs:
        working_pairs.append(oldest_pair)
    
    error_list = []
    dataframes = defaultdict(dict)

    binance = ccxt.binance()
    get_backtest_historical_partial = partial(get_backtest_historical, binance, path=path)

    for pair in working_pairs:
        for tf in timeframes:
            try:
                df = get_backtest_historical_partial(pair, tf)
                dataframes[tf][pair] = df.loc[:]
                del(df)
            except Exception as e:
                print(f" error on {pair} : {e} ")
                error_list.append(pair)
                break

    gc.collect()
    print("Data load 100% use dataframes['1d']['BTC/USDT'] for example to access")
    return dataframes, error_list

dataframes, error_list = get_historical_dataframes(WORKING_PAIRS)


 error on BTG/USDT : [Errno 2] No such file or directory: './database/DataBackTest/1m/BTG-USDT.csv' 
 error on EPS/USDT : [Errno 2] No such file or directory: './database/DataBackTest/1m/EPS-USDT.csv' 
 error on DNT/USDT : [Errno 2] No such file or directory: './database/DataBackTest/1m/DNT-USDT.csv' 
 error on UST/BUSD : [Errno 2] No such file or directory: './database/DataBackTest/1m/UST-BUSD.csv' 
 error on BTG/BUSD : [Errno 2] No such file or directory: './database/DataBackTest/1m/BTG-BUSD.csv' 
 error on EPS/BUSD : [Errno 2] No such file or directory: './database/DataBackTest/1m/EPS-BUSD.csv' 
 error on DNT/BUSD : [Errno 2] No such file or directory: './database/DataBackTest/1m/DNT-BUSD.csv' 
Data load 100% use dataframes['1d']['BTC/USDT'] for example to access


In [31]:
pair_list = WORKING_PAIRS
#tf = '1m'
oldest_pair = "BTC/USDT"
if oldest_pair not in pair_list: pair_list.append(oldest_pair)
df_list1m = {}
df_list1d = {}
df_list1h = {}
df_list5m = {}
df_list15m = {}

ERROR_LIST=[]
for pair in pair_list:
    try:
        df = get_backtest_historical(ccxt.binance(), pair, '1m', path="./database/")
        df_list1m[pair] = df.loc[:]

        df = get_backtest_historical(ccxt.binance(), pair, '1d', path="./database/")
        df_list1d[pair] = df.loc[:]

        df = get_backtest_historical(ccxt.binance(), pair, '1h', path="./database/")
        df_list1h[pair] = df.loc[:]

        df = get_backtest_historical(ccxt.binance(), pair, '5m', path="./database/")
        df_list5m[pair] = df.loc[:]

        df = get_backtest_historical(
            ccxt.binance(), pair, '15m', path="./database/")
        df_list15m[pair] = df.loc[:]
        del(df)
        gc.collect()
    except Exception as e:
        print(f" error on {pair} : {e} ")
        ERROR_LIST.append(pair)

df_list = df_list1m
print("Data load 100% use df_list1d[\"BTC/USDT\"] for exemple to access")



 error on UST/USDT : [Errno 2] No such file or directory: './database/DataBackTest/1m/UST-USDT.csv' 
 error on BTG/USDT : [Errno 2] No such file or directory: './database/DataBackTest/1m/BTG-USDT.csv' 
 error on EPS/USDT : [Errno 2] No such file or directory: './database/DataBackTest/1m/EPS-USDT.csv' 
 error on DNT/USDT : [Errno 2] No such file or directory: './database/DataBackTest/1m/DNT-USDT.csv' 
 error on UST/BUSD : [Errno 2] No such file or directory: './database/DataBackTest/1m/UST-BUSD.csv' 
 error on BTG/BUSD : [Errno 2] No such file or directory: './database/DataBackTest/1m/BTG-BUSD.csv' 
 error on EPS/BUSD : [Errno 2] No such file or directory: './database/DataBackTest/1m/EPS-BUSD.csv' 
 error on DNT/BUSD : [Errno 2] No such file or directory: './database/DataBackTest/1m/DNT-BUSD.csv' 


NameError: name 'prerr' is not defined

In [41]:
ERROR_LIST

['UST/USDT',
 'BTG/USDT',
 'EPS/USDT',
 'DNT/USDT',
 'UST/BUSD',
 'BTG/BUSD',
 'EPS/BUSD',
 'DNT/BUSD']

In [44]:
#!pip install psutil  # install psutil package if not already installed

import psutil

# Get the memory usage of the Jupyter Notebook process
process = psutil.Process()
memory_info = process.memory_info()

# Print the memory usage in MB
print(f"Memory usage: {memory_info.rss / 1024 / 1024:.2f} MB")


Memory usage: 4357.03 MB


In [47]:
def get_min_max_dates(dataframes):
    min_start_date = None
    max_end_date = None

    for tf, df_dict in dataframes.items():
        for pair, df in df_dict.items():
            start_date = df.index.min()
            end_date = df.index.max()

            if min_start_date is None or start_date < min_start_date:
                min_start_date = start_date

            if max_end_date is None or end_date > max_end_date:
                max_end_date = end_date

    return min_start_date, max_end_date

min_start_date, max_end_date = get_min_max_dates(dataframes)
print(f"Minimum start date: {min_start_date}")
print(f"Maximum end date: {max_end_date}")


Minimum start date: 2022-11-11 00:00:00
Maximum end date: 2023-02-22 05:18:00


In [46]:
def get_max_min_dates(dataframes):
    max_start_date = None
    min_end_date = None

    for tf, df_dict in dataframes.items():
        for pair, df in df_dict.items():
            start_date = df.index.min()
            end_date = df.index.max()

            if max_start_date is None or start_date > max_start_date:
                max_start_date = start_date

            if min_end_date is None or end_date < min_end_date:
                min_end_date = end_date

    return max_start_date, min_end_date

max_start_date, min_end_date = get_max_min_dates(dataframes)
print(f"Maximum start date: {max_start_date}")
print(f"Minimum end date: {min_end_date}")


Maximum start date: 2022-11-11 00:00:00
Minimum end date: 2022-11-28 00:00:00


In [49]:
start_period=max_start_date
end_period=min_end_date